In [1]:
from data import *
from transformer import *

# twitterData, twitterTokenizers = loadTwitterThreadData('twitterThreads/')
# inputVocabSize = len(twitterTokenizers[0].wordMap)
# targetVocabSize = len(twitterTokenizers[1].wordMap)
# print()
# print("Vocabulary size:", inputVocabSize, ", ", targetVocabSize)

twitterData, conversationData, tokenizers = loadAllConversations('twitterThreads/')
inputVocabSize = len(tokenizers[0].wordMap)
targetVocabSize = len(tokenizers[1].wordMap)
print("Vocabulary size (Input:", inputVocabSize, "Output:", str(targetVocabSize) + ")")

Conversations: 13118
Twitter threads: 8939
Vocabulary size (Input: 28553 Output: 16406)


In [ ]:
# conversationData, conversationTokenizers = loadConversationData()
# inputVocabSize = len(conversationTokenizers[0].wordMap)
# targetVocabSize = len(conversationTokenizers[1].wordMap)

In [ ]:
# data, tokenizers = loadStableData(100)
#
# inputVocabSize = tokenizers.pt.get_vocab_size().numpy()
# targetVocabSize = tokenizers.en.get_vocab_size().numpy()

In [2]:
numLayers = 4
embeddingDepth = 128
feedDepth = 512
numHeads = 8

optimizer = tf.keras.optimizers.Adam(CustomSchedule(embeddingDepth), beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

model = Transformer(numLayers=numLayers, inputVocabSize=inputVocabSize, targetVocabSize=targetVocabSize, embeddingDepth=embeddingDepth,
                    feedDepth=feedDepth, numHeads=numHeads)
model.compile(loss=masked_loss, optimizer=optimizer, metrics=[masked_accuracy])

In [3]:
model.fit(conversationData, epochs=40)
model.fit(twitterData, epochs=20)

Epoch 1/40


ResourceExhaustedError: Graph execution error:

Detected at node 'transformer/decoder/decoder_layer/causal_self_attention/multi_head_attention_4/einsum/Einsum' defined at (most recent call last):
    File "C:\Users\rapto\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\rapto\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\rapto\PycharmProjects\Ash\venv\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "C:\Users\rapto\PycharmProjects\Ash\venv\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "C:\Users\rapto\PycharmProjects\Ash\venv\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "C:\Users\rapto\PycharmProjects\Ash\venv\lib\site-packages\tornado\pl

In [ ]:
# from datetime import datetime
# now = datetime.now()
# timeText = now.strftime("%y.%m.%d.%H.%M.%S")
# model.export("model " + timeText)

In [ ]:
from transformer import Chat
chat = Chat(tokenizers, model)

In [ ]:
while True:
    prompt = input("Chat with Twitter: ")
    print(prompt)
    output = chat(prompt)
    print(output)